In [7]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetV2S # prot err
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
IMG_SIZE = (300, 300)
BATCH = 24
NUM_CLASSES = 12

train_data_gener = ImageDataGenerator(rescale = 1./255, rotation_range = 20, 
                                width_shift_range = 0.1, height_shift_range = 0.1,
                                zoom_range = 0.2, brightness_range = [0.5, 2],
                                shear_range = 0.2, horizontal_flip = True)
test_data_gener = ImageDataGenerator(rescale = 1./255)
train_data = train_data_gener.flow_from_directory('train', target_size = IMG_SIZE, 
                                                  batch_size = BATCH, class_mode = 'categorical',
                                                  shuffle = True)
test_data = test_data_gener.flow_from_directory('test', target_size = IMG_SIZE, 
                                                  batch_size = BATCH, class_mode = 'categorical',
                                                  shuffle = True)

base_model = EfficientNetV2S(weights='imagenet', include_top=False, input_shape=(*IMG_SIZE,3))
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
outputs = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=outputs)

model.compile(optimizer=Adam(learning_rate=1e-3), loss='categorical_crossentropy',
    metrics=['accuracy'])

history = model.fit(train_data, validation_data=test_data, epochs=10)

test_loss, test_acc = model.evaluate(test_data)
print(f"Test Accuracy: {test_acc*100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")

Found 4395 images belonging to 12 classes.
Found 1099 images belonging to 12 classes.
Epoch 1/10
  5/184 ━━━━━━━━━━━━━━━━━━━━ 6:50 2s/step - accuracy: 0.1031 - loss: 2.4712